In [ ]:
import deepcrunch

In [2]:
from transformers import GPT2Tokenizer, GPT2Model

In [5]:
tokenizer = GPT2Tokenizer.from_pretrained('gpt2-large')
model = GPT2Model.from_pretrained('gpt2-large')

In [6]:
text = "Testing a model for everyone."
encoded_input = tokenizer(text, return_tensors='pt')

In [8]:
output = model(**encoded_input)